# JSON Schema

A JSON schema declares the expected format of a JSON document.

Regular JSON is human-readable, but it doesn't include any context or metadata. JSON Schema enables information about the meanings of the keys and the possible values they can take.

It is a popular method for documenting and validating JSON files.

For more check the [documentation](https://json-schema.org/docs) of JSON schema.

A JSON Schema is a standard used to define and validate the structure and data types of a JSON document (the instance). It is itself a JSON document where the structure and constraints for each **property** are described using predefined **schema keywords** (e.g., type, properties).

For examples chema:

In [20]:
echo '{"type": "number"}' > /tmp/schema.json

Specifies the the root level of the JSON should be a number, e.g. JSON by itself a number.

In [21]:
echo 10 > /tmp/object.json
check-jsonschema --schemafile /tmp/schema.json /tmp/object.json
true

ok -- validation done


The following cell specifies that the root object is a mapping. The available attributes of the root mapping are specified in the `properties` of the JSON schema.

In [23]:
cat << EOF > /tmp/schema.json
{
    "type": "object",
    "properties":
    {
        "wow": {
            "type": "string"
        }
    }
}
EOF
echo '{"wow": "value"}' > /tmp/object.json
check-jsonschema --schemafile /tmp/schema.json /tmp/object.json

ok -- validation done


## Type

The `type` keyword is used to specify the type of the attribute.

Check more in [Type-specified Keywords](https://json-schema.org/understanding-json-schema/reference/type).

---

The following cell indicates that the root element of the JSON should be a string.

In [36]:
echo '{"type": "string"}' > /tmp/schema.json

Specifying a string works fine.

In [38]:
echo '"some string"' > /tmp/object.json
check-jsonschema --schemafile /tmp/schema.json /tmp/object.json
true

ok -- validation done


However, any other object, such as number, will result in the corresponding error.

In [40]:
echo 10 > /tmp/object.json
check-jsonschema --schemafile /tmp/schema.json /tmp/object.json
true

Schema validation errors were encountered.
  /tmp/object.json::$: 10 is not of type 'string'
